# ML Model Horse Racing Predictions
#### Create an ML model that can predict whether a horse has a good chance of winning a race, based on the horse's features

## Preprocessing

In [11]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
cleaned_data_df = pd.read_csv("Data_Racing/International_Data/cleaned_data_df.csv")
cleaned_data_df.head()

,Unnamed: 0,ages,metric,ncond,age,saddle,decimalPrice,isFav,RPR,TR,OR,father,weight,res_win
0,1,4yo+,3218.0,10,6.0,7.0,0.444444,1,96.0,89.0,110.0,Getaway,65,0.0
1,26,5yo+,3218.0,5,8.0,1.0,0.266667,1,115.0,33.0,109.0,Beneficial,73,0.0
2,27,5yo+,3218.0,5,9.0,2.0,0.125000,0,108.0,29.0,103.0,Beneficial,68,0.0
3,43,4yo+,4827.0,6,7.0,13.0,0.076923,0,95.0,39.0,83.0,Kalanisi,63,1.0
4,44,4yo+,4827.0,6,8.0,2.0,0.076923,0,119.0,64.0,108.0,Ask,71,0.0


In [48]:
cleaned_data_df = cleaned_data_df.drop(columns = "Unnamed: 0")

In [49]:
# Determine the number of unique values in each column.
cleaned_data_df.nunique()

ages             16
metric           52
ncond            13
age              14
saddle           29
decimalPrice     79
isFav             2
RPR             165
TR              157
OR              149
father          327
weight           32
res_win           2
dtype: int64

## Select equal res_win values

In [50]:
# Look at value counts for binning
res_win_counts = cleaned_data_df["res_win"].value_counts()
class_count = res_win_counts.min()
class_count

3169

In [63]:
res_win_equal = pd.concat([cleaned_data_df[cleaned_data_df.res_win==0].sample(n=class_count), 
                           cleaned_data_df[cleaned_data_df.res_win==1].sample(n=class_count)])
squeaky_clean_data = res_win_equal.sort_index()
squeaky_clean_data

,ages,metric,ncond,age,saddle,decimalPrice,isFav,RPR,TR,OR,father,weight,res_win
1,5yo+,3218.0,5,8.0,1.0,0.266667,1,115.0,33.0,109.0,Beneficial,73,0.0
3,4yo+,4827.0,6,7.0,13.0,0.076923,0,95.0,39.0,83.0,Kalanisi,63,1.0
4,4yo+,4827.0,6,8.0,2.0,0.076923,0,119.0,64.0,108.0,Ask,71,0.0
10,4yo+,1005.0,0,5.0,4.0,0.047619,0,91.0,69.0,83.0,Showcasing,58,1.0
16,5yo+,4122.5,10,9.0,1.0,0.090909,0,150.0,56.0,140.0,Dylan Thomas,73,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24905,4yo+,3821.0,5,6.0,2.0,0.222222,0,134.0,40.0,135.0,Jeremy,69,1.0
24908,4yo+,4827.0,5,7.0,8.0,0.200000,0,118.0,60.0,117.0,Dubai Destination,69,0.0
24913,4yo+,4022.0,5,7.0,5.0,0.153846,0,144.0,82.0,129.0,Poliglote,70,1.0
24919,4yo+,4725.5,9,8.0,1.0,0.076923,0,141.0,3.0,139.0,Shantou,75,1.0


## Convert and Standardize Data

In [64]:
# Convert categorical data to numeric with `pd.get_dummies`
converted_data = pd.get_dummies(squeaky_clean_data)
converted_data.head()

,metric,ncond,age,saddle,decimalPrice,isFav,RPR,TR,OR,weight,...,father_War Front,father_Well Chosen,father_Westerner,father_Whipper,father_Winged Love,father_Wootton Bassett,father_Yeats,father_Youmzain,father_Zebedee,father_Zoffany
1,3218.0,5,8.0,1.0,0.266667,1,115.0,33.0,109.0,73,...,0,0,0,0,0,0,0,0,0,0
3,4827.0,6,7.0,13.0,0.076923,0,95.0,39.0,83.0,63,...,0,0,0,0,0,0,0,0,0,0
4,4827.0,6,8.0,2.0,0.076923,0,119.0,64.0,108.0,71,...,0,0,0,0,0,0,0,0,0,0
10,1005.0,0,5.0,4.0,0.047619,0,91.0,69.0,83.0,58,...,0,0,0,0,0,0,0,0,0,0
16,4122.5,10,9.0,1.0,0.090909,0,150.0,56.0,140.0,73,...,0,0,0,0,0,0,0,0,0,0


In [65]:
# Split our preprocessed data into our features and target arrays
y = converted_data['res_win']
X = converted_data.drop(columns=['res_win'])

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [66]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [67]:
input_features = len(X_train_scaled[0])
input_features

333

## Compile, Train and Evaluate the Model

In [68]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])
neurons_hidden_layer1 = 256
neurons_hidden_layer2 = 148
neurons_hidden_layer3 = 76

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = neurons_hidden_layer1, activation="relu", input_dim = input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = neurons_hidden_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units = neurons_hidden_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 256)               85504     
                                                                 
 dense_5 (Dense)             (None, 148)               38036     
                                                                 
 dense_6 (Dense)             (None, 76)                11324     
                                                                 
 dense_7 (Dense)             (None, 1)                 77        
                                                                 
Total params: 134,941
Trainable params: 134,941
Non-trainable params: 0
_________________________________________________________________


In [69]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [70]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs = 250)

Epoch 1/250
149/149 [==============================] - 1s 1ms/step - loss: 0.6321 - accuracy: 0.6444
Epoch 2/250
149/149 [==============================] - 0s 906us/step - loss: 0.5262 - accuracy: 0.7299
Epoch 3/250
149/149 [==============================] - 0s 924us/step - loss: 0.4770 - accuracy: 0.7602
Epoch 4/250
149/149 [==============================] - 0s 915us/step - loss: 0.4469 - accuracy: 0.7742
Epoch 5/250
149/149 [==============================] - 0s 923us/step - loss: 0.4131 - accuracy: 0.7957
Epoch 6/250
149/149 [==============================] - 0s 917us/step - loss: 0.3878 - accuracy: 0.8113
Epoch 7/250
149/149 [==============================] - 0s 921us/step - loss: 0.3608 - accuracy: 0.8235
Epoch 8/250
149/149 [==============================] - 0s 923us/step - loss: 0.3406 - accuracy: 0.8378
Epoch 9/250
149/149 [==============================] - 0s 934us/step - loss: 0.3150 - accuracy: 0.8449
Epoch 10/250
149/149 [==============================] - 0s 933us/step - los

149/149 [==============================] - 0s 946us/step - loss: 0.0166 - accuracy: 0.9935
Epoch 81/250
149/149 [==============================] - 0s 977us/step - loss: 0.0488 - accuracy: 0.9832
Epoch 82/250
149/149 [==============================] - 0s 971us/step - loss: 0.0618 - accuracy: 0.9811
Epoch 83/250
149/149 [==============================] - 0s 984us/step - loss: 0.0399 - accuracy: 0.9853
Epoch 84/250
149/149 [==============================] - 0s 950us/step - loss: 0.0407 - accuracy: 0.9880
Epoch 85/250
149/149 [==============================] - 0s 918us/step - loss: 0.0326 - accuracy: 0.9886
Epoch 86/250
149/149 [==============================] - 0s 955us/step - loss: 0.0311 - accuracy: 0.9903
Epoch 87/250
149/149 [==============================] - 0s 997us/step - loss: 0.0122 - accuracy: 0.9962
Epoch 88/250
149/149 [==============================] - 0s 982us/step - loss: 0.0110 - accuracy: 0.9960
Epoch 89/250
149/149 [==============================] - 0s 940us/step - loss:

149/149 [==============================] - 0s 945us/step - loss: 7.3302e-04 - accuracy: 0.9996
Epoch 159/250
149/149 [==============================] - 0s 973us/step - loss: 7.6879e-04 - accuracy: 0.9996
Epoch 160/250
149/149 [==============================] - 0s 1000us/step - loss: 8.0177e-04 - accuracy: 0.9998
Epoch 161/250
149/149 [==============================] - 0s 996us/step - loss: 9.6766e-04 - accuracy: 0.9996
Epoch 162/250
149/149 [==============================] - 0s 972us/step - loss: 6.3385e-04 - accuracy: 0.9996
Epoch 163/250
149/149 [==============================] - 0s 947us/step - loss: 0.0011 - accuracy: 0.9996
Epoch 164/250
149/149 [==============================] - 0s 976us/step - loss: 8.9028e-04 - accuracy: 0.9996
Epoch 165/250
149/149 [==============================] - 0s 980us/step - loss: 7.0850e-04 - accuracy: 0.9996
Epoch 166/250
149/149 [==============================] - 0s 937us/step - loss: 6.6532e-04 - accuracy: 0.9994
Epoch 167/250
149/149 [=============

149/149 [==============================] - 0s 950us/step - loss: 6.2920e-04 - accuracy: 0.9998
Epoch 236/250
149/149 [==============================] - 0s 950us/step - loss: 4.6814e-04 - accuracy: 0.9998
Epoch 237/250
149/149 [==============================] - 0s 942us/step - loss: 3.8336e-04 - accuracy: 0.9996
Epoch 238/250
149/149 [==============================] - 0s 942us/step - loss: 3.2875e-04 - accuracy: 0.9998
Epoch 239/250
149/149 [==============================] - 0s 940us/step - loss: 3.3579e-04 - accuracy: 0.9998
Epoch 240/250
149/149 [==============================] - 0s 944us/step - loss: 3.5762e-04 - accuracy: 0.9996
Epoch 241/250
149/149 [==============================] - 0s 945us/step - loss: 3.3915e-04 - accuracy: 0.9996
Epoch 242/250
149/149 [==============================] - 0s 958us/step - loss: 3.0129e-04 - accuracy: 0.9998
Epoch 243/250
149/149 [==============================] - 0s 953us/step - loss: 2.6318e-04 - accuracy: 0.9998
Epoch 244/250
149/149 [==========

In [71]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

50/50 - 0s - loss: 3.5639 - accuracy: 0.7735 - 94ms/epoch - 2ms/step
Loss: 3.563941240310669, Accuracy: 0.7735015749931335


In [72]:
(y_test == 0).mean()

0.510410094637224

In [73]:
# Make predictions on the test data
y_pred = nn.predict(X_test_scaled)
y_pred_binary = (y_pred > 0.5).astype(int)

50/50 [==============================] - 0s 535us/step


In [74]:
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred_binary)
print("Precision:", precision)

Precision: 0.7564575645756457


In [75]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred_binary))

              precision    recall  f1-score   support

         0.0       0.79      0.76      0.77       809
         1.0       0.76      0.79      0.77       776

    accuracy                           0.77      1585
   macro avg       0.77      0.77      0.77      1585
weighted avg       0.77      0.77      0.77      1585



In [ ]:
# Export our model to HDF5 file
# nn.save('h5_files/AlphabetSoup_Model4.h5')